# Knowledge and Data 2020: Practical Assignment 3 
## RDF Data, RDFS knowledge and inferencing 

YOUR NAME: Michael Moor

YOUR VUNetID: mmr497

*(If you do not provide your name and VUNetID we will not accept your submission).* 

### Learning objectives

At the end of this exercise you should be able to:

1. Access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as [YASGUI](https://yasgui.triply.cc/), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

Before you start, you need to:

- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*
- **Install the free edition of the GraphDB Triplestore:** please follow this short [GraphDB tutorial](https://github.com/raadjoe/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment-3. 

## Task 1: (3.5 points) Integrate Local and External Data

You can integrate SPARQL queries into your Python code by using the *RDFLib* and *SPARQLWrapper* libraries. 

The following code accesses the DBPedia knowledge graph using its SPARQL endpoint, and returns the result of the SPARQL query requesting all the labels asserted to Amsterdam (test it!)  

In [1]:
# This code only works if you are online

from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?cityName
    WHERE { 
        <http://dbpedia.org/resource/Amsterdam> rdfs:label ?cityName 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["cityName"]["value"])  

Amsterdam
أمستردام
Amsterdam
Ámsterdam
Amsterdam
Amsterdam
アムステルダム
Amsterdam
Amsterdam
Amesterdão
Амстердам
阿姆斯特丹


Your task is now the following:
1. Write a SPARQL query that extracts all the cities from your local knowledge graph (constructed by loading the file example-from-slides.ttl) 
2. Find the number of inhabitants of these cities and the longitude and latitude information (if available) from DBPedia.
3. Merge the triples from example-from-slides.ttl with the information extracted from DBpedia + Save all these triples into a new file 'extended-example.ttl' + Print all triples in Turtle Syntax.

For your convenience, we already wrote the following functions that might be useful to complete this task. 
In addition, we have loaded and printed the 'example-from-slides.ttl' dataset.

In [2]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

g = Graph()



# Loads the data from a certain file given as input in Turtle syntax into the Graph g  
# -------------------------
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle')
        

# Prints a certain graph given as input in Turtle syntax
# -------------------------
def serialize(myGraph):
     print(myGraph.serialize(format='turtle').decode("utf-8"))
        

# Saves the Graph g in Turtle syntax to a certain file given as input
# -------------------------
def save(myGraph, filename):
    with open(filename, 'w') as f:
        myGraph.serialize(filename, format='turtle')
        
    
# Changes the namespace of a certain URI given as input to a DBpedia URI 
# Example: transformToDBR("http://example.com/kad2020/Amsterdam") returns "http://dbpedia.org/resource/Amsterdam"
# -------------------------
def transformToDBR(uri):
    if isinstance(uri, Literal):
        # changes the literal to uppercase so that the object with the same name refers to an object and not the string
        return uri.upper()
    components = g.namespace_manager.compute_qname(uri)
    return "http://dbpedia.org/resource/%s"%(components[2])

# -------------------------

load('example-from-slides.ttl')
serialize(g)


# Don't forget to run this cell before continuing the task.


@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City ;
    rdfs:subPropertyOf ex:contains .

ex:Capital rdfs:subClassOf ex:City .

ex:City rdfs:subClassOf ex:Location .

ex:Country rdfs:subClassOf ex:Location .




### 1. Write a SPARQL query that finds all the cities in the dataset

As there is no explicit class City, you will have to find those cities in the dataset (example-from-slides.ttl) using implicit information that can be deduced from the domain and ranges of the relations (e.g. things in a hasCapital relation are capitals and a capital is a city, etc.).

Save all the cities returned from the SPARQL query into the empty set "cities". 

In [3]:
cities = set()

qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?city
        WHERE {
             {?s ex:hasCapital ?city .}
             UNION
             {?s ex:containsCity ?city .}
             UNION
             {?s ex:has_Capital ?city .}

        }
        LIMIT 10

       """)

for row in qres:
    cities.add("%s" % row)
    
for city in cities:
    print(city) 
    

http://example.com/kad2020/Rotterdam
http://example.com/kad2020/Amsterdam
http://example.com/kad2020/Berlin


### 2. For each city, find from DBpedia its longitude & latitude, and its number of inhabitants (if available)

Don't forget to adapt the namespace of the cities in your dataset when querying DBpedia, using the above function *transformToBDR(uri)*

The empty graph g2 should only contain the triples extracted from DBpedia, but added to the URIs with the 'ex' namespace. 
An example of a triple in g2 is the following triple: 
       
       ex:Amsterdam dbo:populationTotal "842343" .

In [4]:
g2 = Graph()

ex = Namespace("http://example.com/kad2020/")
dbo = Namespace("http://dbpedia.org/ontology/")
geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

g2.bind('ex',ex)
g2.bind('dbo',dbo)
g2.bind('geo',geo)


for city in cities:
    city = transformToDBR(city)

    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT ?longtitude ?latitude ?inhabitants
        WHERE { 
            <%s> geo:long ?longtitude ;
            geo:lat ?latitude .
            OPTIONAL {<%s> dbo:populationTotal ?inhabitants .}
            
        }
    """%(city,city))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    city = city.split('/')[-1]
    for result in results["results"]["bindings"]:
        g2.add((URIRef(ex + city), geo.long, Literal(result(["longtitude"]["value"]))))
        g2.add((URIRef(ex + city), geo.lat, Literal(result(["latitude"]["value"]))))
        try:
            g2.add(URIRef((ex + city), dbo.populationTotal, Literal(result(["inhabitants"]["value"]))))
        except:
            pass

serialize(g2)


SyntaxError: invalid syntax (<ipython-input-4-3728b83f0bfb>, line 32)

### 3. Save your results

- Merge the triples from example-from-slides.ttl with the information extracted from DBpedia
- Save all these triples into a new file 'extended-example.ttl'
- Print all triples in Turtle Syntax.


In [5]:
# Your code here



## Task 2: (2.5 points)  Implement Basic Inferencing Rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on rdfs:domain or infer relationships between subjects and objects based on rdfs:subPropertyOf. 

Create rules (or 1 rule?!) to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

We implemented the rdfs2 rule. You should implement the 5 following remaining rules:  

*     (rdfs2) If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)
*     (rdfs3) If G contains the triples (aaa rdfs:range xxx.) and (uuu aaa vvv.) then infer the triple (vvv rdf:type xxx .)
*     (rdfs5) If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple
(uuu rdfs:subPropertyOf xxx.) 
*     (rdfs7) If G contains the triples (aaa rdfs:subPropertyOf bbb.) and (uuu aaa yyy.) then infer the triple (uuu bbb yyy) 
*     (rdfs9) If G contains the triples (uuu rdfs:subClassOf xxx.) and (vvv rdf:type uuu.) then infer the triple
 (vvv rdf:type xxx.)   -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (uuu rdfs:subClassOf vvv.) and (vvv rdfs:subClassOf xxx.) then infer the triple
(uuu rdfs:subClassOf xxx.)


Run your rule reasoner on your knowledge graph.

In [6]:
def myRDFSreasoner(myGraph):
    inferredTriples = 0
    for sbj, prd, obj in myGraph:

        # --- rdfs2 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#domain"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s,o in generator:
                inferredTriples += 1
                print("(rdfs 2) ", s, "rdf:type", obj)
        
        
        # --- rdfs3 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#range"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s,o in generator:
                inferredTriples += 1
                print("(rdfs 3) ", o, "rdf:type", obj)
        
        
        # --- rdfs5 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.subject_objects(URIRef(prd))
            for s,o in generator:
                if obj == s:
                    inferredTriples += 1
                    print("(rdfs 5) ", sbj, "rdfs:subPropertyOf", o)
        
        
        # --- rdfs7 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s,o in generator:
                inferredTriples += 1
                print("(rdfs 7) ", s, obj, o)
         
        
        # --- rdfs9 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.subject_objects(URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"))
            for s,o in generator:
                if sbj == o:
                    inferredTriples += 1
                    print("(rdfs 9) ", s, "rdf:type", obj)
        
        
        # --- rdfs11 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.subject_objects(URIRef(prd))
            for s,o in generator:
                if obj == s:
                    inferredTriples += 1
                    print("(rdfs 11) ", sbj, "rdfs:subClassOf", o)
        
    print("---------------------------------")
    print("Number of inferred triples:", inferredTriples)
    print("---------------------------------")
    
myRDFSreasoner(g)



(rdfs 3)  http://example.com/kad2020/Berlin rdf:type http://example.com/kad2020/Capital
(rdfs 2)  http://example.com/kad2020/Netherlands rdf:type http://example.com/kad2020/Country
(rdfs 3)  http://example.com/kad2020/Germany rdf:type http://example.com/kad2020/Location
(rdfs 3)  http://example.com/kad2020/Rotterdam rdf:type http://example.com/kad2020/City
(rdfs 5)  http://example.com/kad2020/hasCapital rdfs:subPropertyOf http://example.com/kad2020/contains
(rdfs 7)  http://example.com/kad2020/Germany http://example.com/kad2020/containsCity http://example.com/kad2020/Berlin
(rdfs 9)  http://example.com/kad2020/Amsterdam rdf:type http://example.com/kad2020/City
(rdfs 11)  http://example.com/kad2020/Capital rdfs:subClassOf http://example.com/kad2020/Location
(rdfs 7)  http://example.com/kad2020/Netherlands http://example.com/kad2020/closeBy http://example.com/kad2020/Belgium
(rdfs 2)  http://example.com/kad2020/Amsterdam rdf:type http://example.com/kad2020/Location
(rdfs 9)  http://examp

## Task 3: (2 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain (e.g. movies, geography, sports) respecting all the following rules:
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances
*	The instances should be a member of your classes
*	All resources should have an rdfs:label in a suitable language.

You should use (at least) the following language features of RDF and RDFS:
* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

For creating your vocabulary, you can either use a text editor, or add the axioms directly (programatically) to your Knowledge Graph as you did last week. 

Play around with the inference rules you have created in the previous task to make sure that you some added some implicit knowledge, that becomes "visible" via inferencing (this will be useful for the next task). 

Finally:
- Add the knowledge you created into the RDFLIB graph datastructure *myRDFSgraph*, 
- Print *myRDFSgraph* in Turtle so that we can check your "design"
- Save *myRDFSgraph* into a new file 'myRDFSgraph.ttl'

In [7]:
myRDFSgraph = Graph()

rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
ex = Namespace("http://example.com/kad2020/")

myRDFSgraph.bind('ex', ex)
myRDFSgraph.bind('rdf', rdf)
myRDFSgraph.bind('rdfs', rdfs)

myRDFSgraph.add((ex.Game, rdf.type, rdfs.Class))
myRDFSgraph.add((ex.Genre, rdf.type, rdfs.Class))
myRDFSgraph.add((ex.Title, rdf.type, rdfs.Class))
myRDFSgraph.add((ex.DownloadableContent, rdf.type, rdfs.Class))
myRDFSgraph.add((ex.ReleaseData, rdf.type, rdfs.Class))
myRDFSgraph.add((ex.MobileGame, rdfs.subClassOf, ex.Game))
myRDFSgraph.add((ex.ConsoleGame, rdfs.subClassOf, ex.Game))
myRDFSgraph.add((ex.CrossPlatformGame, rdfs.subClassOf, ex.Game))

myRDFSgraph.add((ex.has_title, rdfs.domain, ex.Game))
myRDFSgraph.add((ex.has_title, rdfs.range, ex.Title))
myRDFSgraph.add((ex.has_genre, rdfs.range, ex.Genre))
myRDFSgraph.add((ex.has_genre, rdfs.domain, ex.Game))
myRDFSgraph.add((ex.has_releaseDate, rdfs.domain, ex.Game))
myRDFSgraph.add((ex.has_releaseDate, rdfs.range, ex.ReleaseDate))
myRDFSgraph.add((ex.has_subGenre, rdfs.subPropertyOf, ex.has_genre))
myRDFSgraph.add((ex.has_subGenre, rdfs.range, ex.SubGenre))
myRDFSgraph.add((ex.RolePlayingGame, rdf.type, ex.SubGenre))
myRDFSgraph.add((ex.Survival, rdf.type, ex.SubGenre))
myRDFSgraph.add((ex.Shooter, rdf.type, ex.Genre))
myRDFSgraph.add((ex.PartyGame, rdf.type, ex.Genre))
myRDFSgraph.add((ex.Fantasy, rdf.type, ex.Genre))
myRDFSgraph.add((ex.contains_dlc, rdfs.range, ex.DownloadableContent))
myRDFSgraph.add((ex.DownloadableContent, rdfs.domain, ex.Game))

myRDFSgraph.add((ex.Destiny, rdf.type, ex.CrossPlatformGame))
myRDFSgraph.add((ex.Destiny, ex.has_title, Literal("Destiny")))
myRDFSgraph.add((ex.Destiny, ex.has_genre, ex.Shooter))
myRDFSgraph.add((ex.Destiny, ex.has_subGenre, ex.RolePlayingGame))
myRDFSgraph.add((ex.Destiny, ex.contains_dlc, Literal("Shadowkeep")))
myRDFSgraph.add((ex.Destiny, ex.has_releaseDate, Literal("September 6, 2017")))
myRDFSgraph.add((ex.Destiny, rdfs.label, Literal("Destiny", lang="en")))


myRDFSgraph.add((ex.AmongUs, rdf.type, ex.CrossPlatformGame))
myRDFSgraph.add((ex.AmongUs, ex.has_title, Literal("Among Us")))
myRDFSgraph.add((ex.AmongUs, ex.has_genre, ex.PartyGame))
myRDFSgraph.add((ex.AmongUs, ex.has_subGenre, ex.Survival))
myRDFSgraph.add((ex.AmongUs, ex.contains_dlc, Literal("Pet Bundless")))
myRDFSgraph.add((ex.AmongUs, ex.has_releaseDate, Literal("June 15, 2018")))
myRDFSgraph.add((ex.AmongUs, rdfs.label, Literal("Among Us", lang="en")))

myRDFSgraph.add((ex.Halo5, rdf.type, ex.ConsoleGame))
myRDFSgraph.add((ex.Halo5, ex.has_title, Literal("Halo 5: Gaurdians")))
myRDFSgraph.add((ex.Halo5, ex.has_genre, ex.Shooter))
myRDFSgraph.add((ex.Halo5, ex.contains_dlc, Literal("Memories of Reach")))
myRDFSgraph.add((ex.Halo5, ex.has_releaseDate, Literal("October 27, 2015")))
myRDFSgraph.add((ex.Halo5, rdfs.label, Literal("Halo 5: Gaurdians", lang="en")))

myRDFSgraph.add((ex.RaidShadowLegends, rdf.type, ex.MobileGame))
myRDFSgraph.add((ex.RaidShadowLegends, ex.has_title, Literal("Raid Shadow Legends")))
myRDFSgraph.add((ex.RaidShadowLegends, ex.has_genre, ex.Fantasy))
myRDFSgraph.add((ex.RaidShadowLegends, ex.has_subGenre, ex.RolePlayingGame))
myRDFSgraph.add((ex.RaidShadowLegends, ex.has_releaseDate, Literal("July 29, 2018")))
myRDFSgraph.add((ex.RaidShadowLegends, rdfs.label, Literal("RaidShadowLegends", lang="en")))






print("Now let's check what we can infer from your knowledge graph...")
print("The more rules you cover, the better!")
myRDFSreasoner(myRDFSgraph)
serialize(myRDFSgraph)
save(myRDFSgraph, "myRDFSgraph.ttl")



Now let's check what we can infer from your knowledge graph...
The more rules you cover, the better!
(rdfs 2)  http://example.com/kad2020/Halo5 rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/AmongUs rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/Destiny rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/RaidShadowLegends rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/AmongUs rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/RaidShadowLegends rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/Halo5 rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/Destiny rdf:type http://example.com/kad2020/Game
(rdfs 9)  http://example.com/kad2020/Halo5 rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.com/kad2020/AmongUs rdf:type http://example.com/kad2020/Game
(rdfs 2)  http://example.

## Task 4 (2 points) Compare local inferences with GraphDB results

Upload *myRDFSgraph.ttl* to GraphDB (check [the GraphDB tutorial](https://github.com/raadjoe/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md) before starting to work with GraphDB).

Formulate two different SPARQL queries, and write a Python code that executes these queries over your GraphDB SPARQL endpoint (check example of Task 1).

**Each SPARQL query should return a different type of inferred knowledge** (at least one triple that was not explicitly asserted in the graph).

Specify below next to your query, which type of RDFS rule is the GraphDB reasoner using to infer this answer (rdfs2, rdfs3, rdfs5, rdfs7, rdfs9, rdfs11). 

In [8]:
# Get your GraphDB repository URL and assign it to the variable 'myEndpoint' below. 
# It should be similar (but not the same) to this: 

myEndpoint = "http://145.108.229.246:7200/repositories/assignment-3"
sparql = SPARQLWrapper(myEndpoint)



In [9]:
# Query 1 - Specify which RDFS rule are you testing: 

# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""

""")




In [10]:
# Query 2 - Specify which RDFS rule are you testing: 

# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""

""")

